# 🔥 PyTorch Lightning 

Main notebook for running torch experiments and training torch models


In [1]:
# Autoreload extensions
%load_ext autoreload
%autoreload 2

# Import the package 
import src 

# Display all 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Load datamodule

In [5]:
from src.light.data.comp.datamodule import CompDataModule
from src.data.comp.read_dataset import read_dataframes
from src.aug.apply_transform import comp_transforms

dataframes = read_dataframes()
data_module = CompDataModule(dataframes, comp_transforms)

### Load config for callbacks, trainer and model

In [4]:
from omegaconf import OmegaConf
from termcolor import colored

from src.light.config import (
    CALLBACKS_CONFIG, LOGGERS_CONFIG, TRAINER_CONFIG, 
)
import src.light.trainer

PRINT_ALL = False
if PRINT_ALL: 
    print(colored('------ TRAINER CONFIG ------', 'green'))
    print(colored(f'TRAINER keys: {list(TRAINER_CONFIG.keys())}', 'magenta'))
    print(OmegaConf.to_yaml(TRAINER_CONFIG))

    print(colored('------ CALLBACKS CONFIG ------', 'green'))
    print(colored(f'CALLBACKS keys: {list(CALLBACKS_CONFIG.keys())}', 'magenta'))
    print(OmegaConf.to_yaml(CALLBACKS_CONFIG))

    print(colored('------ LOGGERS CONFIG ------', 'green'))
    print(colored(f'LOGGERS keys: {list(LOGGERS_CONFIG.keys())}', 'magenta'))
    print(OmegaConf.to_yaml(LOGGERS_CONFIG))
    
    
callbacks = src.light.trainer.get_callbacks(CALLBACKS_CONFIG)
loggers = src.light.trainer.get_loggers(LOGGERS_CONFIG)

trainer = src.light.trainer.build_trainer(TRAINER_CONFIG, callbacks, loggers, None)

Instantiating callback <pytorch_lightning.callbacks.BackboneFinetuning>
Instantiating callback <pytorch_lightning.callbacks.EarlyStopping>
Instantiating callback <pytorch_lightning.callbacks.LearningRateMonitor>
Instantiating callback <pytorch_lightning.callbacks.ModelCheckpoint>
Instantiating callback <pl_bolts.callbacks.printing.PrintTableMetricsCallback>


GPU available: False, used: False
TPU available: False, using: 0 TPU cores


Instantiating callback <pl_bolts.callbacks.TrainingDataMonitor>
Instantiating callback <pl_bolts.callbacks.ModuleDataMonitor>
Instantiating callback <pl_bolts.callbacks.BatchGradientVerificationCallback>
Initializing logger <pytorch_lightning.loggers.TensorBoardLogger>
Initializing logger <pytorch_lightning.loggers.WandbLogger>


### MODEL

In [8]:
import src.light.models.model
from torch import nn
backbone = lambda _: 2

FINAL_LAYER = nn.Sequential(
    nn.Linear(backbone.latent_dim, 64), 
    nn.ReLU(),
    nn.Linear(64, data_module.num_classes), 
)


task = src.light.models.model.ClassificationTask(backbone)
trainer.fit(task, data_module)

AttributeError: 'function' object has no attribute 'latent_dim'